# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
#!pip install gmaps

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
# File to Load
WeatherPy_data = "output_data/cities.csv"
WeatherPy_df = pd.read_csv (WeatherPy_data)
WeatherPy_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mataura,-46.19,168.86,51.03,60,70,7.11,NZ,1597106795
1,1,hermanus,-34.42,19.23,44.01,96,0,13.76,ZA,1597106795
2,2,busselton,-33.65,115.33,57.00,91,95,16.28,AU,1597106795
3,3,cape town,-33.93,18.42,57.99,87,20,11.41,ZA,1597106796
4,4,pokhara,28.23,83.98,74.57,75,92,0.47,NP,1597106796


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
gmaps.configure(api_key=g_key)

In [29]:
locations = WeatherPy_df[["Lat", "Lng"]]
#WeatherPy_df = WeatherPy_data.dropna()
humidity = WeatherPy_df["Humidity"]
fig=gmaps.figure()
heat_map=gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=300,point_radius=5)
fig.add_layer(heat_map)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
#max temp less than 80, higher than 70
#wind speed less than 10
#zero cloudiness
#drop if not meeting all 3
IdealWeather_df=WeatherPy_df.loc[(WeatherPy_df["Max Temp"]<80)&\
                                 (WeatherPy_df["Max Temp"]>70)&\
                                 (WeatherPy_df["Wind Speed"]<10)&\
                                 (WeatherPy_df["Cloudiness"]==0)].dropna()
IdealWeather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,kununurra,-15.77,128.73,78.80,28,0,3.36,AU,1597106799
66,66,hami,42.80,93.45,73.35,51,0,9.01,CN,1597106812
76,76,airai,-8.93,125.41,71.26,74,0,2.21,TL,1597106815
90,90,kapoeta,4.77,33.59,72.70,71,0,3.78,SS,1597106819
102,102,jaru,-10.44,-62.47,79.30,40,0,0.72,BR,1597106821


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
hotel_df=IdealWeather_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"]=""
hotel_df

,City,Country,Lat,Lng,Hotel Name
17,kununurra,AU,-15.77,128.73,
66,hami,CN,42.80,93.45,
76,airai,TL,-8.93,125.41,
90,kapoeta,SS,4.77,33.59,
102,jaru,BR,-10.44,-62.47,
161,faanui,PF,-16.48,-151.75,
179,aripuana,BR,-9.17,-60.63,
189,canto do buriti,BR,-8.11,-42.94,
202,erenhot,CN,43.65,111.98,
206,mayumba,GA,-3.43,10.66,


In [31]:
params={
"radius":5000,
"types": "lodging",
"key":g_key}
for index,row in hotel_df.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"]=f"{lat},{lng}"
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    nameaddress=requests.get(base_url,params=params).json()
    #print(nameaddress)
    try:
        hotel_df.loc[index,"Hotel Name"]= nameaddress["results"][0]["name"]
    except:
        print("missing field result skipping")
hotel_df.head()

missing field result skipping
missing field result skipping
missing field result skipping
missing field result skipping
missing field result skipping


,City,Country,Lat,Lng,Hotel Name
17,kununurra,AU,-15.77,128.73,Hotel Kununurra
66,hami,CN,42.80,93.45,Laoyutang Resort
76,airai,TL,-8.93,125.41,Juvinal Martin
90,kapoeta,SS,4.77,33.59,Eastgate Hotel
102,jaru,BR,-10.44,-62.47,Hotel Paraná


In [33]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [35]:
# Add marker layer ontop of heat map
markerlayer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markerlayer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))